# 모델

In [ ]:
# < 라이브러리 임포트 >
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

# ====================================================================================================

# < 데이터셋 경로 >
dataset_path = '/content/drive/MyDrive/Python_project/2023.12.26 프로젝트/data/current_dataset'

# ====================================================================================================

# < 이미지 크기 및 배치 크기 설정 >

# ResNet의 기본 이미지 크기
img_width, img_height = 224, 224
batch_size = 32

# ====================================================================================================

# < 이미지 데이터 생성기 >
train_datagen = ImageDataGenerator(

    # 모든 이미지 픽셀 값을 [0, 255]에서 [0, 1] 범위로 스케일링
    rescale=1./255,

    # 0도에서 10도 사이로 임의의 각도로 이미지를 회전
    rotation_range=10,

    # 이미지를 수평으로 최대 이미지 너비의 10%까지 임의로 이동
    width_shift_range=0.1,

    # 이미지를 수직으로 최대 이미지 높이의 10%까지 임의로 이동
    height_shift_range=0.1,

    # 이미지를 시계 반대 방향으로 0.1 라디안 내외로 변형
    shear_range=0.1,

    # 1에서 10% 줄이거나 늘리는 방식으로 이미지를 확대 또는 축소
    zoom_range=0.1,

    # 이미지를 수평 방향으로 무작위로 뒤집음
    horizontal_flip=True,

    # 이미지를 회전, 이동 또는 줌을 할 때 생기는 빈 픽셀을 인접한 픽셀로 채우는 방법을 지정
    fill_mode='nearest',

    # 전체 데이터셋의 20%를 검증 데이터로 분할
    validation_split=0.2
)

# 위와 같은 파라미터들을 사용하여 모델이 다양한 변형을 가진 이미지에서 작동할 수 있도록
# 훈련 데이터셋의 다양성을 증가시키고 데이터 증강은 과적합을 방지하며 모델의 일반화 능력을 향상시킬 수 있음.

# ====================================================================================================

# < 훈련 및 검증 데이터 생성기 (저장된 이미지 파일들을 신경망 모델에 공급할 준비) >

# 주어진 경로에서 이미지를 로드하여 데이터 생성기를 생성
# 경로에는 여러 하위 폴더가 있고 각 하위 폴더의 이름은 클래스의 레이블로 사용
train_generator = train_datagen.flow_from_directory(

    # 이미지 데이터가 저장된 디렉토리의 경로
    dataset_path,

    # 모델에 입력될 이미지의 크기를 설정 (모든 이미지가 224x224 픽셀 크기로 조정)
    target_size=(img_width, img_height),

    # 한 번에 모델로 전달될 이미지의 수를 설정 (한 배치에 32개)
    batch_size=batch_size,

    # 분류 작업에 대한 레이블을 '원-핫 인코딩' 형식으로 생성하도록 설정
    class_mode='categorical',

    # ImageDataGenerator에 정의된 validation_split 파라미터를 사용하여
    # 데이터셋을 훈련용과 검증용으로 나누는데 사용
    subset='training'# 훈련용
)

# ====================================================================================================

# < 모델의 성능을 검증하는 데 사용되는 이미지 배치를 생성 >
# train_generator와 동일한 옵션
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation' # 검증용
)

# ====================================================================================================

# < ResNet50 모델 불러오기 및 사전 훈련된 가중치 사용 >

# TensorFlow와 Keras 라이브러리를 사용하여 ResNet50 모델을 로드
# weights='imagenet': 모델을 ImageNet 데이터셋으로 사전 훈련된 가중치를 사용하여 초기화
# include_top=False: 네트워크의 최상위 레이어(일반적으로 완전 연결된 레이어)를 포함하지 않겠다는 의미
# input_shape=(img_width, img_height, 3): 네트워크에 공급될 입력 이미지의 shape을 설정
# img_width와 img_height는 이미지의 너비와 높이를 나타내고 3은 이미지의 채널 수(대개 RGB 이미지의 경우 3)를 의미
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# ====================================================================================================

# < 모델 커스터마이징 >

# base_model은 ResNet 모델을 의미
# base_model.output은 이 모델의 출력을 나타냄
# 출력은 추가적인 레이어에 연결될 기본 데이터임
x = base_model.output

# 각 특징 맵(feature map)에 대해 평균값을 계산하여 더 적은 수의 특징으로 정보를 압축함
# 모델의 매개변수 수를 줄이고 과적합을 방지함
x = GlobalAveragePooling2D()(x)

# 1024개의 뉴런을 가진 밀집(Dense) 레이어를 추가
# 활성화 함수로는 하이퍼볼릭 탄젠트(tanh)를 사용
# tanh 함수는 출력값을 -1과 1 사이
x = Dense(1024, activation='tanh')(x)

# 드롭아웃 레이어를 추가하여 훈련 중 무작위로 50%(반절)의 뉴런을 비활성화
# 과적합을 방지
x = Dropout(0.5)(x)

# 512, 256, 128개의 뉴런을 갖는 밀집 레이어를 추가
# tanh 활성화 함수 사용
x = Dense(512, activation='tanh')(x)
x = Dense(256, activation='tanh')(x)
x = Dense(128, activation='tanh')(x)

# 최종 출력층 (분류해야 할 클래스의 수)
# softmax 활성화 함수는 다중 클래스 분류 문제에 적합함
# 각 클래스에 대한 확률 분포를 출력
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# ====================================================================================================

# < 최종 모델 >

# Model은 케라스에서 모델을 정의하는 데 사용되는 클래스
# 클래스를 사용하여 함수형 API를 통해 모델을 생성
# inputs=base_model.input: 모델의 입력으로 사용될 것을 정의
# base_model.input은 로드한 ResNet 모델의 입력을 의미
# outputs=predictions: 모델의 출력으로 사용될 것을 정의
# predictions는 신경망의 최종 출력 레이어 (활성화 함수인 softmax를 사용)
model = Model(inputs=base_model.input, outputs=predictions)

# 위 코드는 사전 훈련된 ResNet 모델에 새로운 레이어를 추가하여
# 사용자 정의 모델을 만드는 과정이며 생성된 model 객체는 훈련, 평가, 예측 등에 사용

# ====================================================================================================

# < 모델 컴파일 > (Keras를 사용하여 신경망 모델을 컴파일하는 과정)

# optimizer=Adam(learning_rate=0.0001): 모델의 최적화 알고리즘을 지정
# Adam 최적화 알고리즘을 사용
model.compile(optimizer=Adam(learning_rate=0.0001),

              # 학습률(learning rate)을 0.0001로 설정
              loss='categorical_crossentropy',

              # metrics=['accuracy']: 학습 과정에서 모니터링할 성능 지표를 지정
              # accuracy(정확도)를 사용
              metrics=['accuracy'])

# 컴파일 단계를 통해 모델은 학습에 필요한 추가 설정을 완료하며
# 이후 모델의 fit 메서드를 통해 실제 학습을 시작
# 위 설정은 모델이 어떻게 학습할지와 학습 과정에서 어떻게 성능을 평가할지를 결정

# ====================================================================================================

# < 체크포인트 저장 경로 >
checkpoint_path = '/content/drive/MyDrive/Python_project/2023.12.26 프로젝트/KWAK/Cross_validation_checkpoint'

# ====================================================================================================

# < 체크포인트 콜백 설정 > (모델을 학습하는 동안 모델의 가중치를 자동으로 저장)
checkpoint_callback = ModelCheckpoint(

    # checkpoint_path: 모델의 가중치를 저장할 파일 경로
    # 학습 중에 가장 좋은 모델의 가중치가 이 경로에 저장
    checkpoint_path,

    # monitor='val_accuracy': 콜백이 모니터링할 성능 지표를 지정
    # 'val_accuracy'를 사용하여 검증 데이터셋에 대한 모델의 정확도를 기준으로 최적의 모델을 판단
    monitor='val_accuracy',

    # verbose=1: 콜백의 진행 상황에 대한 자세한 출력을 활성화
    # 모델의 가중치가 갱신될 때마다 메시지를 출력
    verbose=1,

    # save_best_only=True: 옵션을 True로 설정하면
    # 이전에 저장된 모델보다 더 나은 성능의 모델만 저장
    save_best_only=True,

    # mode='max': monitor로 지정된 지표가 최대값일 때 모델을 저장
    # 'val_accuracy'와 같은 지표의 경우 높은 값이 더 좋은 성능을 의미하므로 'max' 모드를 사용
    mode='max'
)

# 위 코드는 모델이 각 에폭마다 'val_accuracy'를 기준으로
# 현재까지의 최고 성능을 갱신할 때마다 해당 가중치를 checkpoint_path에 저장
# 학습이 끝난 후 최적의 성능을 낸 모델을 로드 할 수 있음

# ====================================================================================================

# < 콜백 설정 >

# monitor='val_loss': 'val_loss' (검증 데이터셋에 대한 손실)를 모니터링
# patience=5: 모델이 5 에폭 동안 'val_loss'에서 개선되지 않으면 학습을 조기에 중단
# verbose=1: 진행 상황에 대한 자세한 정보를 출력
# mode='min': monitor로 지정된 지표가 최소값일 때 조치를 취하도록 지시하며
# 손실의 경우에는 낮은 값이 더 좋은 성능을 의미하므로 'min' 모드를 사용
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')

# 콜백은 모델이 더 이상 개선되지 않을 때 과적합을 방지하기 위해 학습을 조기에 중단
# monitor='val_loss': 'val_loss'를 모니터링
# factor=0.2: 학습률을 감소시킬 때 적용할 감소율이고 현재 학습률에 이 값을 곱하여 새로운 학습률을 계산
# patience=2: 2 에폭 동안 'val_loss'에서 개선이 없을 경우 학습률을 감소
# verbose=1: 진행 상황에 대한 자세한 정보를 출력
# mode='min': monitor로 지정된 지표가 최소값일 때 조치를 취하도록 지시
# min_lr=0.00001: 학습률을 감소시킬 수 있는 최소값을 지정하여 학습률이 이 값 이하로 내려가지 않도록 해줌
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='min', min_lr=0.00001)

# 위 코드는 모델이 손실 함수의 국소 최소점에 빠지지 않고 전역 최소점에 더 가까워질 수 있게 해줌

# ====================================================================================================

# < 모델 학습 >

# model.fit: 모델 학습 시작
history = model.fit(

    # train_generator: 훈련 데이터를 제공하는 데이터 생성기이며 ImageDataGenerator를 사용하여
    # 생성된 train_generator는 디스크에서 이미지를 읽고 적절한 전처리를 수행한 후 모델에 공급
    train_generator,

    # steps_per_epoch=len(train_generator): 한 에폭(epoch) 동안 훈련에 사용할 스텝(배치)의 수를 정의
    # train_generator에서 생성되는 전체 배치의 수와 동일하게 설정
    steps_per_epoch=len(train_generator),

    # epochs=20: 모델이 훈련 데이터셋을 총 20번 반복해서 학습
    epochs=20,

    # validation_data=validation_generator: 검증 데이터를 제공하는 데이터 생성기이며
    # 모델의 성능을 학습 중간에 평가하기 위해 사용
    validation_data=validation_generator,

    # validation_steps=len(validation_generator): 한 에폭 동안 검증에 사용할 스텝(배치)의 수를 정의
    # validation_generator에서 생성되는 전체 배치의 수와 동일하게 설정
    validation_steps=len(validation_generator),

    # callbacks=[checkpoint_callback, early_stopping, reduce_lr]: 콜백(callbacks) 리스트를 정의하고
    # 이전에 정의한 ModelCheckpoint, EarlyStopping, ReduceLROnPlateau 콜백이 포함
    callbacks=[checkpoint_callback, early_stopping, reduce_lr]
)

# ====================================================================================================

# < 모델 평가 >

# model.evaluate 메서드를 사용하여 검증 데이터셋에 대한 모델의 손실(loss)과 정확도(accuracy)를 계산

# model.evaluate: 모델을 평가하는 함수이며
# 주어진 데이터셋에 대해 모델의 손실과 메트릭(정확도)을 계산

# validation_generator: 검증 데이터셋을 제공하는 데이터 생성기이며
# 검증 데이터셋의 이미지를 배치 단위로 모델에 공급

# 반환값인 validation_loss와 validation_accuracy는
# 각각 검증 데이터셋에 대한 모델의 손실과 정확도를 나타냄
validation_loss, validation_accuracy = model.evaluate(validation_generator)

# 검증 손실을 출력
print(f"Validation Loss: {validation_loss}")

# 검증 정확도를 출력
print(f"Validation Accuracy: {validation_accuracy}")

# ====================================================================================================

# < 모델 저장 >

# os.path.join: 여러 경로 구성 요소를 결합하여 하나의 경로를 형성 (확장자 = .h5)
model_save_path = os.path.join('/content/drive/MyDrive/Python_project/2023.12.26 프로젝트/KWAK/model_Res_Net', 'resnet_model.h5')

# model.save: 전체 모델(아키텍처, 가중치, 훈련 구성, 옵티마이저 상태 등)을
# 단일 파일에 저장하여 지정 경로에 모델 파일을 생성
model.save(model_save_path)

# ====================================================================================================

# 교차 검증을 위한 EfficientNetB0 모델 사용
# Res_Net과 동일한 환경과 튜닝 값으로 학습을 진행
# Res_Net과 동일한 코드 (설명 생략)

# < EfficientNetB0 모델 로드 >
base_model_eff = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# < 모델 커스터마이징 >
x = base_model_eff.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='tanh')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='tanh')(x)  # 탄젠트 활성화 함수를 사용하는 추가 밀집 레이어
x = Dense(256, activation='tanh')(x)  # 추가 밀집 레이어
x = Dense(128, activation='tanh')(x)  # 추가 밀집 레이어
predictions = Dense(train_generator.num_classes, activation='softmax')(x)  # 출력층

# < 최종 모델 >
model_eff = Model(inputs=base_model_eff.input, outputs=predictions)

# < 모델 컴파일 >
model_eff.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# < 모델 학습 >
history_eff = model_eff.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[checkpoint_callback, early_stopping, reduce_lr]
)

# < 모델 저장 >
model_save_path_eff = os.path.join('/content/drive/MyDrive/Python_project/2023.12.26 프로젝트/KWAK/model_EfficientNet', 'efficientnet_model.h5')
model_eff.save(model_save_path_eff)


Found 5755 images belonging to 5 classes.
Found 1437 images belonging to 5 classes.
Epoch 1/20
180/180 [==============================] - ETA: 0s - loss: 0.3326 - accuracy: 0.8862

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Epoch 1: val_accuracy improved from -inf to 0.16075, saving model to /content/drive/MyDrive/Python_project/2023.12.26 프로젝트/KWAK/Cross_validation_checkpoint
180/180 [==============================] - 169s 738ms/step - loss: 0.3326 - accuracy: 0.8862 - val_loss: 4.0288 - val_accuracy: 0.1608 - lr: 1.0000e-04
Epoch 2/20
180/180 [==============================] - ETA: 0s - loss: 0.1296 - accuracy: 0.9526

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Epoch 2: val_accuracy improved from 0.16075 to 0.19624, saving model to /content/drive/MyDrive/Python_project/2023.12.26 프로젝트/KWAK/Cross_validation_checkpoint
180/180 [==============================] - 130s 725ms/step - loss: 0.1296 - accuracy: 0.9526 - val_loss: 3.5977 - val_accuracy: 0.1962 - lr: 1.0000e-04
Epoch 3/20
180/180 [==============================] - ETA: 0s - loss: 0.0764 - accuracy: 0.9731

...


Epoch 11: val_accuracy improved from 0.88379 to 0.90257, saving model to /content/drive/MyDrive/Python_project/2023.12.26 프로젝트/KWAK/Cross_validation_checkpoint
180/180 [==============================] - 131s 727ms/step - loss: 0.0088 - accuracy: 0.9979 - val_loss: 0.4461 - val_accuracy: 0.9026 - lr: 2.0000e-05
Epoch 12/20
180/180 [==============================] - ETA: 0s - loss: 0.0073 - accuracy: 0.9983

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Epoch 12: val_accuracy did not improve from 0.90257
180/180 [==============================] - 107s 594ms/step - loss: 0.0073 - accuracy: 0.9983 - val_loss: 0.4649 - val_accuracy: 0.8984 - lr: 2.0000e-05
Epoch 13/20
180/180 [==============================] - ETA: 0s - loss: 0.0054 - accuracy: 0.9991
Epoch 13: val_accuracy did not improve from 0.90257

Epoch 13: ReduceLROnPlateau reducing learning rate to 1e-05.
180/180 [==============================] - 106s 588ms/step - loss: 0.0054 - accuracy: 0.9991 - val_loss: 0.5164 - val_accuracy: 0.8914 - lr: 2.0000e-05
Epoch 14/20
180/180 [==============================] - ETA: 0s - loss: 0.0046 - accuracy: 0.9988
Epoch 14: val_accuracy did not improve from 0.90257
180/180 [==============================] - 107s 595ms/step - loss: 0.0046 - accuracy: 0.9988 - val_loss: 0.4676 - val_accuracy: 0.8970 - lr: 1.0000e-05
Epoch 15/20
180/180 [==============================] - ETA: 0s - loss: 0.0045 - accuracy: 0.9988
Epoch 15: val_accuracy did not imp

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


IsADirectoryError: ignored

# 결과출력 및 시각화

In [ ]:
# < 라이브러리 임포트 >
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# < 테스트 데이터셋에 대한 데이터 생성기(test_generator)를 설정 >

# 테스트 데이터셋이 저장된 디렉토리의 경로
test_dataset_path = '/content/drive/MyDrive/Python_project/2023.12.26 프로젝트/data/test_dataset_Overseas'

# 모든 이미지 픽셀 값을 [0, 255] 범위에서 [0, 1] 범위로 조정 (스케일링)
test_datagen = ImageDataGenerator(rescale=1./255)

# 디스크의 디렉토리로부터 이미지를 읽어 데이터 생성기를 설정
test_generator = test_datagen.flow_from_directory(

    # 이미지를 로드할 디렉토리의 경로
    test_dataset_path,

    # 모든 이미지를 224x224 픽셀 크기로 조정
    target_size=(224, 224),

    # 한 번에 모델로 전달될 이미지의 수를 32로 설정
    batch_size=32,

    # 다중 클래스 분류를 위해 레이블을 '원-핫 인코딩' 형태로 변환
    class_mode='categorical',

    # 테스트 데이터셋의 순서를 유지 (셔플을 사용하지 않음)
    shuffle=False
)


In [ ]:
# < 저장된 모델 로드 >

resnet_model_path = '/content/drive/MyDrive/Python_project/2023.12.26 프로젝트/KWAK/model_Res_Net/resnet_model.h5'
efficientnet_model_path = '/content/drive/MyDrive/Python_project/2023.12.26 프로젝트/KWAK/model_EfficientNet/efficientnet_model.h5'

resnet_model = load_model(resnet_model_path)
efficientnet_model = load_model(efficientnet_model_path)


In [ ]:
# 모델 평가
resnet_results = resnet_model.evaluate(test_generator, verbose=0)
efficientnet_results = efficientnet_model.evaluate(test_generator, verbose=0)

# 정확도 출력
print(f'ResNet Test Accuracy: {resnet_results[1]}')
print(f'EfficientNet Test Accuracy: {efficientnet_results[1]}')

# 예측 및 실제 레이블 비교
test_generator.reset()  # 인덱스 리셋
resnet_predictions = np.argmax(resnet_model.predict(test_generator), axis=1)
test_generator.reset()  # 다시 리셋
efficientnet_predictions = np.argmax(efficientnet_model.predict(test_generator), axis=1)
true_labels = test_generator.classes

# 오분류된 데이터 분석
misclassified_resnet = np.where(resnet_predictions != true_labels)[0]
misclassified_efficientnet = np.where(efficientnet_predictions != true_labels)[0]

# 오분류된 데이터 인덱스 출력
print(f'Misclassified by ResNet: {misclassified_resnet}')
print(f'Misclassified by EfficientNet: {misclassified_efficientnet}')


In [ ]:
# 테스트 데이터셋에 대한 모델의 예측
y_pred_resnet = resnet_model.predict(test_generator)
y_pred_efficientnet = efficientnet_model.predict(test_generator)

# 실제 클래스 레이블
true_classes = test_generator.classes

# 예측된 클래스 레이블
predicted_classes_resnet = np.argmax(y_pred_resnet, axis=1)
predicted_classes_efficientnet = np.argmax(y_pred_efficientnet, axis=1)

# 혼동 행렬 계산
cm_resnet = confusion_matrix(true_classes, predicted_classes_resnet)
cm_efficientnet = confusion_matrix(true_classes, predicted_classes_efficientnet)


In [ ]:
# < matplotlib 한글화 >

import matplotlib.pyplot as plt
!apt-get update -qq
!apt-get install fonts-nanum* -qq
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf',  name='NanumGothic')
fm.fontManager.ttflist.insert(0, fe)
plt.rcParams.update({'font.size': 18, 'font.family': 'NanumGothic'})

In [ ]:
# 클래스 레이블 목록
class_labels = list(test_generator.class_indices.keys())

# ResNet 모델 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(cm_resnet, annot=True, cmap='Reds', fmt='g', xticklabels=class_labels, yticklabels=class_labels)
plt.title('ResNet Model')
plt.xlabel('예측값')
plt.ylabel('실제값')
plt.show()

# EfficientNet 모델 혼동 행렬 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(cm_efficientnet, annot=True, cmap='Blues', fmt='g', xticklabels=class_labels, yticklabels=class_labels)
plt.title('EfficientNet Model')
plt.xlabel('예측값')
plt.ylabel('실제값')
plt.show()


In [ ]:
# 훈련 이력에서 'val_loss' 데이터를 로드
# 에포크 1 ~ 13 까지의 val_loss
resnet_val_loss = history.history['val_loss'][:14]
efficientnet_val_loss = history_eff.history['val_loss'][:14]

# 실제 손실 값의 길이에 맞는 에포크 범위를 설정
epochs_trimmed_resnet = range(1, len(resnet_val_loss) + 1)
epochs_trimmed_efficientnet = range(1, len(efficientnet_val_loss) + 1)

# 두 모델의 검증 손실을 플로팅(시각화)
plt.figure(figsize=(10, 6))
plt.plot(epochs_trimmed_resnet, resnet_val_loss, 'r', marker='o', label='ResNet')
plt.plot(epochs_trimmed_efficientnet, efficientnet_val_loss, 'b', marker='o', label='EfficientNetB0')
plt.title('검증 손실률 비교')
plt.xlabel('Epoch')
plt.xticks(range(1, 15))
plt.ylabel('loss')
plt.legend()
plt.grid(True)
plt.show()
